## EDA of fe dataset (general features dataset)

Examine data from `fe` dataset and select features to work with in the next notebooks.


In [1]:
import pandas as pd
from sklearn.inspection import permutation_importance
from utils import io
from utils import eda
import utils.analyze as analyze
import utils.model_lgb as model_lgb
import project.project_api as project_api
import plotly.express as px

In [2]:
RANDOM_SEED = 42
TARGET_KEY = "target"
CLASS_NUM = 5

train_fe_path = "./data/train_fe"
test_fe_path = "./data/test_fe"
baseline_model_name = "2024_05_31_baseline_fe_acc_4699"

In [3]:
df_train = pd.read_parquet(train_fe_path)

X_train = df_train.drop(columns=[TARGET_KEY])
# lgb wants zero-based categories
y_train = df_train[TARGET_KEY] - 1

df_test = pd.read_parquet(test_fe_path)
X_test = df_test.drop(columns=[TARGET_KEY])
# lgb requires class to be zero-based
y_test = df_test[TARGET_KEY] - 1
del df_test

print(df_train.info(max_cols=1000))
df_train

<class 'pandas.core.frame.DataFrame'>
Index: 146953 entries, 1525928 to 132580172
Data columns (total 816 columns):
 #    Column                            Non-Null Count   Dtype  
---   ------                            --------------   -----  
 0    Ama_rchrgmnt_sum_max_mnt1         146914 non-null  float16
 1    content_clc_mea_mnt1              146914 non-null  float16
 2    content_cnt_max_mnt1              146914 non-null  float16
 3    voice_out_short_part_max_mnt1     146916 non-null  float16
 4    voice_mts_in_nrest_part_std_mnt1  146916 non-null  float16
 5    num_act_days_max_mnt1             146916 non-null  float16
 6    sms_roam_clc_min_mnt1             146914 non-null  float16
 7    voice_in_cmpttrs_avg_durmin_mnt1  146916 non-null  float16
 8    com_num_part_mea_mnt1             146914 non-null  float16
 9    pay_avg_mea_mnt1                  146916 non-null  float16
 10   voice_out_tar_dur_std_mnt1        146902 non-null  float16
 11   voice_out_tar_dur_min_mnt1       

,Ama_rchrgmnt_sum_max_mnt1,content_clc_mea_mnt1,content_cnt_max_mnt1,voice_out_short_part_max_mnt1,voice_mts_in_nrest_part_std_mnt1,num_act_days_max_mnt1,sms_roam_clc_min_mnt1,voice_in_cmpttrs_avg_durmin_mnt1,com_num_part_mea_mnt1,pay_avg_mea_mnt1,...,MV_Traf_ACCA_out_v_Min,MV_Traf_mn_out_v_Min,MV_DOU_OT,MV_SERV_Y_WO_AF,MV_Migr_To,MV_SERV_RLH,MV_DOU_PPM_VF,MV_DOU_Neg_Bal,MV_ot_total,target
abon_id,,,,,,,,,,,,,,,,,,,,,
1525928,0.0,0.0,10.828125,1.000977,1.000000,5.324219,0.0,31.453125,1.226562,25.843750,...,0.000000,0.0,4.210938,NaN,0.0,NaN,12.789062,NaN,0.0,4
1530471,0.0,0.0,9.976562,1.000977,1.000000,5.324219,0.0,0.000000,1.005859,13.328125,...,0.000000,0.0,NaN,NaN,0.0,NaN,17.906250,NaN,NaN,4
1541528,0.0,0.0,12.101562,1.000977,1.000977,5.324219,0.0,10.625000,1.009766,4.980469,...,0.000000,0.0,2.207031,NaN,0.0,NaN,NaN,NaN,0.0,3
1542028,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
1542203,0.0,0.0,0.000000,0.000000,0.000000,1.480469,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,NaN,NaN,0.0,NaN,12.789062,NaN,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132486100,0.0,0.0,6.750000,0.000000,0.000000,5.324219,0.0,0.000000,0.000000,26.171875,...,0.000000,0.0,1.480469,NaN,0.0,NaN,NaN,NaN,0.0,4
132550466,0.0,0.0,4.785156,0.000000,0.000000,5.324219,0.0,0.000000,1.001953,24.390625,...,6.195312,0.0,3.589844,NaN,0.0,NaN,NaN,NaN,0.0,3
132551440,0.0,0.0,2.921875,0.000000,0.000000,5.324219,0.0,0.000000,0.000000,21.093750,...,0.000000,0.0,2.207031,NaN,0.0,NaN,NaN,NaN,0.0,3


In [4]:
eda.print_missing(df_train)
print()

Missing values report:
Features count without missing values: 4

Percent of missing values by feature (features count 812):
                                   Total    Rate
bs_of_unsucc_attemp_equip_m1      146953  100.00
bs_drop_call_rate                 146953  100.00
bs_of_succ_m1                     146953  100.00
bs_succ_rate                      146953  100.00
bs_drop_rate                      146953  100.00
bs_of_recall_m1                   146953  100.00
bs_of_attemps_all_m1              146953  100.00
bs_of_succ_but_drop_m1            146953  100.00
bs_of_unsucc_low_balance_m1       146953  100.00
device_has_gprs                   146953  100.00
bs_recall_rate                    146953  100.00
entertainment                     146947  100.00
MV_FRAUD_BLOCK                    146941   99.99
Food                              146935   99.99
MV_SERV_RLH                       146931   99.99
tsoa_mail_cnt                     146830   99.92
Cars                              146806   

In [5]:
eda.print_uniq(df_train)
print()

Unique values report:
                                   uniq      rate
bs_drop_call_rate                     0  0.000000
bs_succ_rate                          0  0.000000
bs_drop_rate                          0  0.000000
bs_of_recall_m1                       0  0.000000
bs_of_attemps_all_m1                  0  0.000000
device_has_gprs                       0  0.000000
bs_of_unsucc_low_balance_m1           0  0.000000
bs_recall_rate                        0  0.000000
bs_of_unsucc_attemp_equip_m1          0  0.000000
bs_of_succ_but_drop_m1                0  0.000000
bs_of_succ_m1                         0  0.000000
Ama_rchrgmnt_sum_max_mnt1             1  0.000007
goodok_clc_std_mnt3                   1  0.000007
Ama_rchrgmnt_sum_min_mnt1             1  0.000007
Ama_rchrgmnt_sum_mea_mnt1             1  0.000007
Ama_rchrgmnt_sum_td_mnt3              1  0.000007
goodok_clc_max_mnt1                   1  0.000007
Ama_rchrgmnt_sum_max_mnt3             1  0.000007
Ama_rchrgmnt_sum_std_mnt3   

## Category distribution


In [6]:
target_counts = df_train.groupby(TARGET_KEY)[TARGET_KEY].count()

target_labels = {
    1: "<20 years",
    2: "20-30 years",
    3: "30-40 years",
    4: "40-50 years",
    5: ">50 years",
}

df_target_dist = pd.DataFrame(
    {
        "label": target_counts.index.map(lambda x: target_labels[x]),
        "count": target_counts,
        "dataset%": target_counts.apply(lambda x: round(x / len(df_train) * 100, 2)),
    }
).reset_index()

print(df_target_dist)

fig = px.pie(
    df_target_dist,
    values="count",
    names="label",
    title=f"Total records: {len(df_train)}",
)

fig.update_traces(textposition="inside", textinfo="percent+value+label")
fig.show()

   target        label  count  dataset%
0       1    <20 years  18960     12.90
1       2  20-30 years  27428     18.66
2       3  30-40 years  39579     26.93
3       4  40-50 years  31692     21.57
4       5    >50 years  29294     19.93


## Permutation feature importance for baseline model


In [7]:
predict_baseline_model, baseline_model = model_lgb.load(baseline_model_name)


def calc_permutation():
    class LgbAdapter:
        def __init__(self, predict: callable):
            self.do_predict = predict

        def fit(self):
            raise Exception("Unexpected fit call")

        def predict(self, df):
            return self.do_predict(df)

    df_test = pd.read_parquet(test_fe_path)
    X_test = df_test.drop(columns=[TARGET_KEY])
    # lgb requires class to be zero-based
    y_test = df_test[TARGET_KEY] - 1

    p_importance_result = permutation_importance(
        LgbAdapter(predict_baseline_model),
        X_test,
        y_test,
        scoring=["accuracy"],
        n_jobs=-1,
        random_state=RANDOM_SEED,
    )

    return pd.concat(
        [
            pd.DataFrame(
                {
                    "feature": X_test.columns,
                    "mean": p_importance_result["accuracy"]["importances_mean"],
                    "std": p_importance_result["accuracy"]["importances_std"],
                },
            ),
            pd.DataFrame(
                p_importance_result["accuracy"]["importances"],
                columns=["run1", "run2", "run3", "run4", "run5"],
            ),
        ],
        axis=1,
    )


p_imp_df = io.run_cached("./data/fe_permutation_importance.parquet", calc_permutation)

p_imp_df["importance_tree"] = baseline_model.feature_importance()
p_imp_df["p_importance%"] = p_imp_df["mean"] / p_imp_df["mean"].sum() * 100
p_imp_df["importance_tree%"] = (
    p_imp_df["importance_tree"] / p_imp_df["importance_tree"].sum() * 100
)

# count how much rows (subscribers) have values per feature to get abon_count per feature
p_imp_df["abon_count"] = X_train.agg(lambda series: series[series > 0].count()).values
p_imp_df["coverage"] = p_imp_df["abon_count"] / len(X_train)
p_imp_df = p_imp_df.sort_values(by="mean", ascending=False)

In [8]:
p_imp_df.nlargest(30, columns=["mean"])

,feature,mean,std,run1,run2,run3,run4,run5,importance_tree,p_importance%,importance_tree%,abon_count,coverage
606,lt,0.061995,0.000931,0.063015,0.062202,0.060510,0.061412,0.062834,307,22.742929,3.231579,146350,0.995897
648,imei_mean_days_usage,0.024439,0.000720,0.023473,0.025278,0.025233,0.024330,0.023879,191,8.965357,2.010526,146580,0.997462
651,imei_mean_day_announced,0.007322,0.000560,0.008351,0.007313,0.007200,0.006658,0.007087,155,2.685964,1.631579,139756,0.951025
622,myvf_day_usage,0.007191,0.000893,0.008373,0.005936,0.006523,0.007945,0.007177,187,2.637941,1.968421,132692,0.902955
322,voice_in_mts_avg_dur_mea_mnt3,0.006681,0.000294,0.006681,0.006162,0.006929,0.006636,0.006997,96,2.450818,1.010526,114229,0.777317
639,imei_mean_price,0.005281,0.000468,0.005304,0.005236,0.005056,0.004695,0.006116,135,1.937471,1.421053,141325,0.961702
655,device_brand_apple,0.005114,0.000417,0.004965,0.004582,0.004898,0.005326,0.005800,86,1.876201,0.905263,19578,0.133226
654,imei_mean_long_days_usage,0.004528,0.000778,0.004514,0.005755,0.004559,0.003295,0.004514,117,1.660926,1.231579,146511,0.996992
103,conn_out_uniq_cnt_max_mnt3,0.004401,0.001039,0.002821,0.004785,0.004514,0.003905,0.005981,52,1.614559,0.547368,110180,0.749764
562,voice_in_mts_avg_dur_min_mnt3,0.003977,0.000200,0.003882,0.003814,0.004130,0.004288,0.003769,68,1.458899,0.715789,79775,0.542861


In [9]:
px.scatter(
    p_imp_df,
    x="p_importance%",
    y="importance_tree%",
    color="coverage",
    hover_data=["feature"],
    color_continuous_scale="matter",
)

In [10]:
def visualize_p_imp():
    df = p_imp_df[p_imp_df["mean"] > 0].sort_values(by="mean", ascending=False)

    fig = px.bar(
        df,
        x="feature",
        y="p_importance%",
        color="coverage",
        title=f"Prediction contribution by feature. Features count: {len(df)}",
        hover_data=["abon_count", "mean", "std", "importance_tree"],
        width=1000,
    )
    fig.show()

    df_coverage = df[(df["coverage"] >= 0.01) | (df["p_importance%"] >= 0.01)]

    fig = px.bar(
        df,
        x="feature",
        y="coverage",
        color="p_importance%",
        title=f"Subscriber coverage by feature. Features count: {len(df_coverage)}",
        hover_data=["abon_count", "mean", "std", "importance_tree"],
        width=1000,
    )
    fig.show()


visualize_p_imp()

In [11]:
def plot_p_imp():
    df = p_imp_df[p_imp_df["p_importance%"] >= 0.5].sort_values(
        by="mean", ascending=False
    )
    df = pd.DataFrame(
        df[["run1", "run2", "run3", "run4", "run5"]].T.values,
        columns=df["feature"].to_list(),
    )

    return px.box(
        df,
        x=df.columns.to_list(),
        height=500,
    )
    # df.plot.box(vert=False, whis=10)


plot_p_imp()

In [12]:
predict_baseline, model_baseline = model_lgb.load("2024_05_31_baseline_fe_acc_4699")

In [13]:
def train_topk_features(k: float):
    df_topk = p_imp_df.nlargest(int(k), columns=["mean"])["feature"]
    feature_count = len(df_topk)

    print(f"\n\n=== Training on top {feature_count} features ===")

    X_train_pruned = X_train[df_topk.values]
    X_test_pruned = X_test[df_topk.values]

    predict_pruned, model_pruned = model_lgb.train_multiclass(
        X_train=X_train_pruned,
        y_train=y_train,
        X_test=X_test_pruned,
        y_test=y_test,
        # use params from one of the latest optuna runs
        params={
            "eta": 0.19997109376050565,
            "boosting_type": "gbdt",
            "lambda_l1": 5.735491139313952e-07,
            "lambda_l2": 3.1791646476628225e-06,
            "num_leaves": 20,
            "min_data_in_leaf": 30,
            "feature_fraction": 0.8166299199026185,
            "bagging_fraction": 0.9867527250605056,
            "bagging_freq": 7,
            "verbosity": -1,
        },
        num_class=CLASS_NUM,
        seed=RANDOM_SEED,
    )

    print(f"Train dataset for {len(df_topk)} features:")
    report_train = project_api.report(
        y_test=y_train,
        y_pred=predict_pruned(X_train_pruned),
    )

    print(f"\nTest dataset for {len(df_topk)} features:")
    report_test = project_api.report(
        y_test=y_test,
        y_pred=predict_pruned(X_test_pruned),
    )

    return (feature_count, report_train["accuracy"], report_test["accuracy"])


def profile_feature_counts():
    result = {
        "feature_count": [],
        "accuracy_train": [],
        "accuracy_test": [],
    }
    total_features = len(p_imp_df)

    for topk_value in [
        total_features * 1,
        total_features * 0.75,
        total_features * 0.5,
        total_features * 0.25,
        150,
        100,
        75,
        50,
        40,
        20,
        10,
    ]:
        feature_count, accuracy_train, accuracy_test = train_topk_features(topk_value)

        result["feature_count"].append(feature_count)
        result["accuracy_train"].append(accuracy_train)
        result["accuracy_test"].append(accuracy_test)

    return pd.DataFrame(result)


df_feature_count_profile = io.run_cached(
    "./data/fe_feature_count_vs_acc.parquet", profile_feature_counts
)
df_feature_count_profile

,feature_count,accuracy_train,accuracy_test
0,815,0.562466,0.468052
1,611,0.558151,0.468120
2,407,0.558947,0.467714
3,203,0.552905,0.465367
4,150,0.548202,0.464915
5,100,0.543235,0.466089
6,75,0.538295,0.465908
7,50,0.529891,0.458799
8,40,0.523535,0.456090
9,20,0.488054,0.437312


In [14]:
px.line(
    df_feature_count_profile,
    x="feature_count",
    y=["accuracy_train", "accuracy_test"],
    title="Accuracy vs feature count",
    labels={"feature_count": "Feature acount", "value": "Accuracy"},
    markers=".",
    log_x=True,
)

On the graph above we can see a plateau from 75 till 815, therefore the most value is in the first 75 features. We will select those features to use in the composite dataset in next notebooks.


## Export models and feature data


In [15]:
top75_features_path = "./data/fe_top75_features.json"
io.write_json(
    top75_features_path,
    p_imp_df.nlargest(75, columns=["mean"])["feature"].to_list(),
)

In [16]:
def train_model_on_top_features(
    study_name: str,
    feature_selection_path: str,
):
    features = io.read_json(feature_selection_path)

    df_train = pd.read_parquet(train_fe_path)
    X_train = df_train[features]
    # lgb wants zero-based categories
    y_train = df_train[TARGET_KEY] - 1
    del df_train

    print(f"Train X: {X_train.shape}")
    print(f"Train y: {y_train.shape}")

    df_test = pd.read_parquet(test_fe_path)
    X_test = df_test[features]
    # lgb requires class to be zero-based
    y_test = df_test[TARGET_KEY] - 1
    del df_test

    print(f"Test X: {X_test.shape}")
    print(f"Test y: {y_test.shape}")

    study = project_api.train_lgb(
        study_name=study_name,
        X_train=X_train,
        y_train=y_train,
        X_test=X_test,
        y_test=y_test,
    )

    predict_fe, model_fe = model_lgb.train_multiclass(
        X_train=X_train,
        y_train=y_train,
        X_test=X_test,
        y_test=y_test,
        params=study.best_params,
        num_class=CLASS_NUM,
        seed=RANDOM_SEED,
        name=f"2024_06_24_{study_name}",
    )

    print("Train dataset:")
    project_api.report(
        y_test=y_train,
        y_pred=predict_fe(X_train),
    )

    print("\n\nTest dataset:")
    project_api.report(
        y_test=y_test,
        y_pred=predict_fe(X_test),
    )

In [17]:
# Best hyperparameters:  {'boosting_type': 'dart', 'eta': 0.38999660826917953, 'num_leaves': 27, 'min_data_in_leaf': 20, 'feature_fraction': 0.9021404218699807, 'bagging_fraction': 0.9608019633467612, 'bagging_freq': 7, 'lambda_l1': 5.6991908634402e-05, 'lambda_l2': 0.06470315155225144}
# Best score:  0.46690139255648094
train_model_on_top_features(
    study_name="fe_top75_acc_4659",
    feature_selection_path=top75_features_path,
)

Train X: (146953, 75)
Train y: (146953,)
Test X: (44307, 75)
Test y: (44307,)


[I 2024-06-24 22:24:01,486] Using an existing study with name 'fe_top75_acc_4659' instead of creating a new one.


Best hyperparameters:  {'boosting_type': 'dart', 'eta': 0.38999660826917953, 'num_leaves': 27, 'min_data_in_leaf': 20, 'feature_fraction': 0.9021404218699807, 'bagging_fraction': 0.9608019633467612, 'bagging_freq': 7, 'lambda_l1': 5.6991908634402e-05, 'lambda_l2': 0.06470315155225144}
Best score:  0.46690139255648094
[LightGBM] [Warning] Ignoring unrecognized parameter 'early_stopping_min_delta' found in model string.
Train dataset:
Accuracy: 0.5422550067028233


Test dataset:
Accuracy: 0.46690139255648094


In [18]:
analyze.multiclass_report(
    y_true_tr=y_train,
    y_proba_tr=predict_baseline(X_train, proba=True),
    y_true_val=y_test,
    y_proba_val=predict_baseline(X_test, proba=True),
    report=True,
)

Metrics         Train      Test       Δ         
roc_auc         0.8479     0.7939     -0.054    
accuracy        0.5604     0.4699     -0.0905   
precision       0.5604     0.4699     -0.0905   
recall          0.5604     0.4699     -0.0905   
f1_score        0.5604     0.4699     -0.0905   


Train:
              precision    recall  f1-score   support

           0       0.61      0.71      0.66     18960
           1       0.56      0.53      0.55     27428
           2       0.52      0.60      0.56     39579
           3       0.55      0.36      0.44     31692
           4       0.60      0.65      0.62     29294

    accuracy                           0.56    146953
   macro avg       0.57      0.57      0.56    146953
weighted avg       0.56      0.56      0.55    146953

Test:
              precision    recall  f1-score   support

           0       0.54      0.63      0.59      5582
           1       0.47      0.45      0.46      8264
           2       0.43      0.51      